<a href="https://colab.research.google.com/github/israelkingz/umojahack_classifier/blob/main/update(feature_engineering_and_data_balancing)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# dataframe and plotting
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# machine learning
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import mean_absolute_error

In [ ]:
# Load files into a pandas dataframe /content/drive/MyDrive/214892-1.xlsx
df = pd.read_csv('/content/drive/MyDrive/Train.csv')
test = pd.read_csv('/content/drive/MyDrive/Test.csv')
ss = pd.read_csv('/content/drive/MyDrive/SampleSubmission.csv')

In [ ]:
df.head()

,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_1,Yes,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,2018,uniqueid_2,No,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,2018,uniqueid_3,Yes,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Kenya,2018,uniqueid_4,No,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Kenya,2018,uniqueid_5,No,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed


In [ ]:
test.head()

,country,year,uniqueid,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_6056,Urban,Yes,3,30,Male,Head of Household,Married/Living together,Secondary education,Formally employed Government
1,Kenya,2018,uniqueid_6060,Urban,Yes,7,51,Male,Head of Household,Married/Living together,Vocational/Specialised training,Formally employed Private
2,Kenya,2018,uniqueid_6065,Rural,No,3,77,Female,Parent,Married/Living together,No formal education,Remittance Dependent
3,Kenya,2018,uniqueid_6072,Rural,No,6,39,Female,Head of Household,Married/Living together,Primary education,Remittance Dependent
4,Kenya,2018,uniqueid_6073,Urban,No,3,16,Male,Child,Single/Never Married,Secondary education,Remittance Dependent


In [ ]:
# Group education_level
df['education_level'] = df['education_level'].replace(['Other/Dont know/RTA'], 'Unknown')
df['education_level'] = df['education_level'].replace(['Primary education', '6'], 'Primary')
df['education_level'] = df['education_level'].replace(['Secondary education', 'Secondary / O level'], 'Secondary')
df['education_level'] = df['education_level'].replace(['Tertiary education', 'Vocational/Specialised training'], 'Tertiary')

# Group job_type
df['job_type'] = df['job_type'].replace(['Self employed', 'Self Employed'], 'self-employed')
df['job_type'] = df['job_type'].replace(['Government Dependent', 'Formally employed Private', 'Formally employed Government'], 'skilled')
df['job_type'] = df['job_type'].replace(['Informally employed', 'Informally employed: Agriculture', 'Informally employed: Hotel work', 'Informally employed: Manufacturing', 'Informally employed: Other'], 'unskilled')

# Convert categorical columns to numerical using one-hot encoding
cat_cols = ['country', 'year', 'location_type', 'cellphone_access', 'gender_of_respondent', 'relationship_with_head', 'marital_status', 'education_level', 'job_type']
df = pd.get_dummies(df, columns=cat_cols)


In [ ]:
# Drop uniqueid column
#test.drop(['uniqueid'], axis=1, inplace=True)

# Group education_level
test['education_level'] = test['education_level'].replace(['Other/Dont know/RTA'], 'Unknown')
test['education_level'] = test['education_level'].replace(['Primary education', '6'], 'Primary')
test['education_level'] = test['education_level'].replace(['Secondary education', 'Secondary / O level'], 'Secondary')
test['education_level'] = test['education_level'].replace(['Tertiary education', 'Vocational/Specialised training'], 'Tertiary')

# Group job_type
test['job_type'] = test['job_type'].replace(['Self employed', 'Self Employed'], 'self-employed')
test['job_type'] = test['job_type'].replace(['Government Dependent', 'Formally employed Private', 'Formally employed Government'], 'skilled')
test['job_type'] = test['job_type'].replace(['Informally employed', 'Informally employed: Agriculture', 'Informally employed: Hotel work', 'Informally employed: Manufacturing', 'Informally employed: Other'], 'unskilled')

# Convert categorical columns to numerical using one-hot encoding
cat_cols = ['country', 'year', 'location_type', 'cellphone_access', 'gender_of_respondent', 'relationship_with_head', 'marital_status', 'education_level', 'job_type']
test = pd.get_dummies(test, columns=cat_cols)


In [ ]:
# Split the data into features and target
# Cobvert target label to numerical Data
le = LabelEncoder()
df['bank_account'] = le.fit_transform(df['bank_account'])

X = df.drop(['bank_account', 'uniqueid'], axis=1)
y = df['bank_account']

# Scale the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Balance the dataset using SMOTE
smote = SMOTE()
X, y = smote.fit_resample(X, y)

In [ ]:
# Split train_data
from sklearn.model_selection import train_test_split

X_Train, X_Val, y_Train, y_val = train_test_split(X, y, stratify = y, 
                                                  test_size = 0.2, random_state=42)

In [ ]:
#import classifier algorithm here
from xgboost import XGBClassifier

# create models
xg_model = XGBClassifier()

from sklearn.metrics import confusion_matrix, accuracy_score

# fit by setting best parameters and Evaluate model
xgb_model = XGBClassifier(min_child_weight=1, gamma=1, subsample=0.8, max_depth=5)

xgb_model.fit(X_Train, y_Train)
y_pred = xgb_model.predict(X_Val)

# Get error rate
print("Error rate of the XGB classifier: ", 1 - accuracy_score(y_val, y_pred))

Error rate of the XGB classifier:  0.08212739641311073


More steps


Ensembling of models


Cross-validation: Group folds, Stratified

In [ ]:
# Define models to try out
models = [LogisticRegression(), RandomForestClassifier(), GradientBoostingClassifier()]

# Define cross-validation strategies
group_kfold = GroupKFold(n_splits=5)
stratified_kfold = StratifiedKFold(n_splits=5)

In [ ]:
for model in models:
  mae_scores = []
for train_idx, test_idx in stratified_kfold.split(X, y):
  X_train, X_test = X[train_idx], X[test_idx]
  y_train, y_test = y[train_idx], y[test_idx]
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  mae_scores.append(mean_absolute_error(y_test, y_pred))
print(f'{str(model)} MAE: {np.mean(mae_scores)}')

GradientBoostingClassifier() MAE: 0.3137060860959921


In [ ]:
from sklearn.ensemble import VotingClassifier
ensemble = VotingClassifier(estimators=[('lr', LogisticRegression()), ('rf', RandomForestClassifier()), ('gb', GradientBoostingClassifier())], voting='soft')
mae_scores = []
for train_idx, test_idx in stratified_kfold.split(X, y):
  X_train, X_test = X[train_idx], X[test_idx]
  y_train, y_test = y[train_idx], y[test_idx]
  ensemble.fit(X_train, y_train)
  y_pred = ensemble.predict(X_test)
  mae_scores.append(mean_absolute_error(y_test, y_pred))
print(f'Ensemble MAE: {np.mean(mae_scores)}')

Ensemble MAE: 0.22355707862741156
